# Создание метода защиты персональных данных клиентов страховой компании «Хоть потоп»

______________________
Нам необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.
______________________

______

#### Данное исследование разделим на несколько частей.
______________________
### Часть 1: Загрузка данных
* [1. Изучение файла с данными, получение общей информации, загрузка библиотек.](#1-bullet)

### Часть 2: Умножение признаков на обратимую матрицу
* [1. Примеры матириц и их влияние на качество модели.](#2-bullet)

### Часть 3: Алгоритм преобразования данных
* [1. Написание аглоритма преобразование и обоснование его эффективности.](#3-bullet)

### Часть 4: Проверка алгоритма
* [1. Проверка качества линейной регрессии из sklearn на основе метрики r2.](#4-bullet)
____________________

#### 1-bullet
### Часть 1: Загрузка данных
1. Изучение файла с данными, получение общей информации, загрузка библиотек.

In [1]:
# загрузим необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# прочитаем файл и получим общие данные
df = pd.read_csv('/datasets/insurance.csv')
print(df.info())
print()
print(df.describe())
print()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None

               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  5000.000000  5000.000000   5000.000000  5000.000000        5000.000000
mean      0.499000    30.952800  39916.360000     1.194200           0.148000
std       0.500049     8.440807   9900.083569     1.091387           0.463183
min       0.000000    18.000000   5300.000000     0.000000           0.000000
25%       0.000000    24.000000  33300.000000     0.000000           0.000000
50%       0.000000    30.000000  40200.000000     1.000000           0.000000
75%       1.000000    37.000000  46600.000000     2.000000           0.000000
max

- получили таблицу с 5 столбцами, длиной 5000 строк, пропусков не имеется, все данные численного типа. данные корректны и пригодны для исследования.

#### 2-bullet
### Часть 2: Умножение признаков на обратимую матрицу
1. Примеры матириц и их влияние на качество модели.

In [3]:
# получим и сохраним признаки и целевой признак
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']

Создадим класс модели, где:

найдем вектор предсказаний по формуле 𝑎=𝑋𝑤 + 𝑤0

Задача обучения найти вектор весов:

𝑤=argmin𝑤𝑀𝑆𝐸(𝑋𝑤,𝑦)

Обучать будем по следующей формуле:

𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦

In [4]:
# создадим класс модели
class MyLinearRegression:
    # обучим
    def fit(self, train_features, train_target):
        # сохраним матрицу Х, состоющую из признков
        # куда добавим единичный столбец
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        # сохраним вектор значений для целевого признака
        y = train_target
        # найдем вектора весов
        w = ((np.linalg.inv(X.T.dot(X))).dot(X.T)).dot(y)
        # получим вектор весов
        self.w = w[1:]
        # получим вектор сдвига
        self.w0 = w[0]

    # получим предсказания
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

создадим функцию, которая принимает на вход признаки и целевой признак, а возвращает качество нашей модели.

In [5]:
# создадим функцию
def get_r2_LR(features, target):
    # объявим модель, обучим и получим предсказания
    model = MyLinearRegression()
    model.fit(features, target)
    predictions = model.predict(features)
    # найдем метрику r2
    print('Качество модели', r2_score(target, predictions))

In [6]:
# применим функцию
get_r2_LR(features, target)

Качество модели 0.42494550286668


Далее займемся умножением признаков на матрицу. Для этого сначала создадим функцию для провеки матриц на обратимость.

In [7]:
def check_matrix(matrix):
    check = matrix
    Inv_check = np.linalg.inv(check)
    if np.allclose(np.dot(check, Inv_check), np.eye(check.shape[0])) is True:
        answer = "обратимая"
    else:
        answer = "необратимая"
    return print('матрица', answer)

Создадим квадратную случайную матрицу и проверим ее на обратимость.

In [8]:
# создадим случайную матрицу 
P = np.random.normal(size=(4, 4))

# применим функцию
check_matrix(P)

матрица обратимая


Получили обратимую матрицу, теперь умножим признаки на обратимую матрицу и проверим качество нашей модели по вновь образованным признакам (поскольку признаки - это вектор, а произведение вектора на матрицу равно вектору).

In [9]:
# умножим признаки на матрицу
features_1 = features.dot(P)

# применим функцию
get_r2_LR(features_1, target)

Качество модели 0.42494550286652166


**Ответ:** 

- Качество линейной регрессии не изменилось.

**Обоснование:** 

- Причина по которой, качество линейной регрессии не изменилось заключается в том, что признаки умножили именно на обратимую матрицу. При обучении, когда рассчитываем вектор весов, мы пользуемся следующей формулой 𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦. Здесь в указанную формулу вместо матрицы Х мы подставляем произведение Х^Р (Р можно вынести за скобки). при этом помним, что матрица Р у нас обратимая, а произведение матрицы на обратную равно единичной матрице, т.е. единице. Поэтому в результате умножения не меняется взаимное расположение векторов. 

#### 3-bullet
### Часть 3: Алгоритм преобразования данных
1. Написание аглоритма преобразование и обоснование его эффективности.

**Алгоритм**

Создадим функцию, которая будет принимать на вход признаки (векторы). В этой же функции создадим случайную квадратную матрицу и перемножим признаки на полученную обратимую матрицу. На выходе получим преобразованные признаки.  

In [10]:
def conv_f(features):
    # создадим случайную матрицу 
    P = np.random.normal(size=(4, 4))
    # проверим на обратимость
    check_matrix(P)
    # умножим признаки на матрицу
    new_features = features.dot(P)
    return new_features

выведем для наглядности на экран исходные и преобразованные данные.

In [11]:
# выведем на экран исходные данные
print(features.head())
print()
# выведем на экран преобразованные данные
print(conv_f(features).head())

   Пол  Возраст  Зарплата  Члены семьи
0    1     41.0   49600.0            1
1    0     46.0   38000.0            1
2    0     29.0   21000.0            0
3    0     21.0   41700.0            2
4    1     28.0   26100.0            0

матрица обратимая
              0             1             2             3
0  20896.092539 -22507.321558  27219.314562 -98057.279687
1  15985.195668 -17251.505124  20854.973519 -75128.939551
2   8827.696803  -9535.562046  11525.756641 -41519.558749
3  17591.511851 -18917.676781  22883.487297 -82435.272015
4  10984.417941 -11845.705244  14323.182066 -51600.630314


**Обоснование**

Таким образом, чтобы зашифровать исходные данные можно умножить признаки на обратимую матрицу.

#### 4-bullet 
### Часть 4: Проверка алгоритма
1. Проверка качества линейной регрессии из sklearn на основе метрики r2.

создадим функцию, которая на вход принимает признаки и целевой признак, затем за основу берет модель линейной регрессии из sklearn, обучает эту модель, получает предсказания и возвращает метрику r2.

In [12]:
# создадим функцию для проверки качества
# модели из sklearn
def get_r2LR(features, target):
    model = LinearRegression()
    model.fit(features, target)
    predictions = model.predict(features) 
    result = r2_score(target, predictions)
    return print('Качество модели:', result)

In [13]:
# применим функцию для изначальных данных
get_r2LR(features, target)

Качество модели: 0.42494550286668


Далее преобразуем признаки путем умножения на случайную обратимую матрицу и найдем качество.

In [14]:
# получим и сохраним зашиврованные признаки
conv_features = conv_f(features)

# применим функцию проверки качества
get_r2LR(conv_features, target)

матрица обратимая
Качество модели: 0.4249455028666851


- Получили такое же качество модели с преобразованными данными, как и с исходными данными.